In [1]:
import pandas as pd
from myUtils import list_dump
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
### edit file name to Barchart 52-week_new-highs ####
path_fn_52wkHi = "C:/Users/ping/Downloads/all-us-exchanges-52-week-new-highs-09-28-2022.csv"
#######################

In [2]:
df = pd.read_csv(path_fn_52wkHi)  # read Barchart 52-week_new-highs
# drop last row (e.g. Downloaded from Barchart.com as of 08-24-2022 06:21pm CDT)
df.drop(df.tail(1).index,inplace=True)
df_shape_orig = df.shape
# create column of Last * Volume
df['Last*Vol'] = df.Last * df.Volume

In [3]:
# apply filter Last*Vol > $10mil, Last > $10
df = df[df['Last*Vol'] > 10000000]   # $10mil
# df = df[df['Last*Vol'] > 5000000]  # $5 mil 
df = df[df['Last'] > 10] 
df_shape_after = df.shape
row_deleted = df_shape_orig[0] - df_shape_after[0]
print(f'number of symbols removed by filter Last*Vol > $10M: {row_deleted}')
# print(f'number of symbols removed by filter Last*Vol > $5M:  {row_deleted}')
print(f'number of symbols remain:                            {df_shape_after[0]}')

number of symbols removed by filter Last*Vol > $10M: 11
number of symbols remain:                            7


In [4]:
# write symbols to text file
symbols_52wkHi = df['Symbol'].to_list()
list_dump(symbols_52wkHi, path_data_dump, 'symbols_52wkHi.txt', verbose=True)


+ def list_dump(list_to_dump, path_list_dump, filename)

Wrote list to C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_52wkHi.txt
- def list_dump(list_to_dump, path_list_dump, filename)

------------------------------------------------------------------------------



In [5]:
# columns for df_52wkHi
myCols = ['symbol',
 'quoteType',
 'longName',
 'sector',
 'industry',
 'marketCap',
 'revenueGrowth',
 'ebitdaMargins',
 'earningsGrowth',
 'earningsQuarterlyGrowth',
 'previousClose',
 'volume',
 'fiftyTwoWeekHigh',
 '52WeekChange',
 'revenuePerShare',
 'forwardEps',
 'trailingEps',
 'sharesOutstanding'
]

In [6]:
from datetime import date
# put today's date in the date column
today = str(date.today())
df['date'] = today

In [7]:
import yfinance as yf
# get yf.Ticker(symbol).info for keys in myCols for 52wkHi symbols 
df_list = []
for symbol in symbols_52wkHi:
    df_list.append(pd.DataFrame([yf.Ticker(symbol).info]))
df = pd.concat(df_list)
df = df[myCols].copy()

In [8]:
from myUtils import pickle_dump
# sort df_52wkHi with highest revenueGrowth on the 1st row
df_52wkHi = df.sort_values(by=['revenueGrowth', 'ebitdaMargins', 'earningsQuarterlyGrowth', 'earningsGrowth' ], ascending=False)
pickle_dump(df_52wkHi, path_data_dump, 'df_52wkHi')

In [9]:
# apply filter revenueGrowth > .50 & ebitdaMargins > .25
df_filtered = df_52wkHi.query('revenueGrowth > .50 & ebitdaMargins > .25')
df_filtered


,symbol,quoteType,longName,sector,industry,marketCap,revenueGrowth,ebitdaMargins,earningsGrowth,earningsQuarterlyGrowth,previousClose,volume,fiftyTwoWeekHigh,52WeekChange,revenuePerShare,forwardEps,trailingEps,sharesOutstanding
0,AZPN,EQUITY,"Aspen Technology, Inc.",Technology,Software—Application,14898728960,2.089,0.30418,None,None,232.67,272382,240.0,0.86285,7.486,7.64,3.99,62168700


In [10]:
symbols_52wkHi = df_filtered['symbol'].tolist()  # convert pd.Series to list
pickle_dump(symbols_52wkHi, path_data_dump, 'symbols_52wkHi')

In [11]:
# plot symbols
# symbols_52wkHi = ["ARLP", "STNG"]
from myUtils import yf_candlestick
plot_days = '252d'
for symbol in symbols_52wkHi:
  df = yf.Ticker(symbol).history(period=plot_days)
  df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)  # drop Div. and Stock Splits columns
  df.columns = ['open', 'high', 'low', 'close', 'volume']  # rename columns Open ... Volume to open ... volume
  cache_plot = yf_candlestick(symbol, df, plot_chart=True)

In [12]:
# print symbols data
from myUtils import yf_print_symbol_data
cache_print = yf_print_symbol_data(symbols_52wkHi)

AZPN
                  Open        High         Low       Close  Volume  Dividends  Stock Splits
Date                                                                                       
2022-09-22  223.520004  224.570007  220.449997  223.309998  162900          0             0
2022-09-23  220.500000  222.440002  216.809998  222.149994  209700          0             0
2022-09-26  221.050003  225.139999  219.750000  221.479996  225100          0             0
2022-09-27  224.619995  234.020004  222.649994  232.669998  413100          0             0
2022-09-28  233.529999  240.000000  229.190002  239.649994  272382          0             0

symbol                    AZPN
quoteType                 EQUITY
longName                  Aspen Technology, Inc.
industry                  Software—Application
sector                    Technology
marketCap                     14.899B
revenueGrowth                  2.089
earningsGrowth            None
earningsQuarterlyGrowth   None
ebitdaMargins   